In [1]:
import pamtra2
import collections
import numpy as np
import pandas as pn
import xarray as xr
from collections import OrderedDict
from copy import deepcopy, copy
import toolz
import collections
import matplotlib.pyplot as plt

try:
    %matplotlib inline
except:
    pass

# create Profile

In [2]:
additionalDims = collections.OrderedDict()
additionalDims['time'] = pn.date_range('2016-01-01','2016-01-05',freq='D')[:1]
nHeights = 10

pam2 = pamtra2.pamtra2(
    nLayer=nHeights,
    hydrometeors=['rain','snow'],
    additionalDims = additionalDims,
    frequencies = [35e9, 94e9],
)



In [3]:
pam2.profile

<xarray.customProfile>
Dimensions:              (frequency: 2, hydrometeor: 2, layer: 10, time: 1)
Coordinates:
  * time                 (time) datetime64[ns] 2016-01-01
  * layer                (layer) int64 0 1 2 3 4 5 6 7 8 9
  * hydrometeor          (hydrometeor) <U4 'rain' 'snow'
  * frequency            (frequency) float64 3.5e+10 9.4e+10
Data variables:
    height               (time, layer) float64 nan nan nan nan nan nan nan ...
    temperature          (time, layer) float64 nan nan nan nan nan nan nan ...
    pressure             (time, layer) float64 nan nan nan nan nan nan nan ...
    relativeHumidity     (time, layer) float64 nan nan nan nan nan nan nan ...
    horizontalWind       (time, layer) float64 nan nan nan nan nan nan nan ...
    verticalWind         (time, layer) float64 nan nan nan nan nan nan nan ...
    eddyDissipationRate  (time, layer) float64 nan nan nan nan nan nan nan ...
    hydrometeorContent   (time, layer, hydrometeor) float64 nan nan nan nan ...
    

Fill with some dummie values

In [4]:

pam2.profile.height[:] = np.linspace(0,1000,nHeights)
pam2.profile.temperature[:] = 250 
pam2.profile.relativeHumidity[:] = 90
pam2.profile.pressure[:] = 100000
pam2.profile.eddyDissipationRate[:] = 1e-4
pam2.profile.horizontalWind[:] = 0



pam2.profile.hydrometeorContent.values[:] = 0
#rain
pam2.profile.hydrometeorContent.values[...,5:8,0] = 1e-4
#snow
pam2.profile.hydrometeorContent.values[...,5:8,1] = 2e-4



AttributeError: 'customProfile' object has no attribute 'waterContent'

For testing, dask arrays are turned on by chunking.

In [ ]:
pam2.profile = pam2.profile.chunk({'time':1, 'hydrometeor':1, 'frequency':1})

In [ ]:
pam2.addMissingVariables()
pam2.profile 

In [ ]:
# # OR EASIER if desired: 
# pam2 = pamtra2.importers.profiles.usStandardAtmosphere(heigths)

# Describe Hydrometeors
To describe the hydrometeors, you can use functions, xr.DataArrays or scalars. Teh functions must support xr.DataArrays. The kwargs of the functions are gathered from (in this order)

1. pam2.hydrometors.name.profile variables (name must match). E.g. density (after it has been calculated) 
2. pam2.profile variables (name must match). E.g. temperature
3. Other provided kwargs (e.g. Dmin, DMax, N0 etc)
4. Default function arguments. Can be turned of with useFuncArgDefaults = False to make sure everything is explicitly covered.



In [ ]:

pam2.describeHydrometeor(
    pamtra2.hydrometeors.softEllipsoidFixedDensity,
    name = 'rain', #or None, then str(index)
    nBins =40,
    sizeBounds = pamtra2.hydrometeors.size.linspaceBounds, 
    sizeCenter = pamtra2.hydrometeors.size.boundsToMid,
    sizeBoundsWidth = pamtra2.hydrometeors.size.boundsWidth,
    sizeDistribution = pamtra2.hydrometeors.sizeDistribution.exponentialN0WC, 
    aspectRatio = 1.0,
    mass = pamtra2.hydrometeors.mass.ellipsoid,
    density = pamtra2.hydrometeors.density.water,
    crossSectionArea = pamtra2.hydrometeors.crossSectionArea.sphere,
    relativePermittivity = pamtra2.libs.refractiveIndex.water.turner_kneifel_cadeddu, #replace with refractiveIndex.water.Turner.n
    scattering = pamtra2.hydrometeors.scattering.Rayleigh,
    fallVelocity = pamtra2.hydrometeors.fallVelocity.khvorostyanov01_drops,
    Dmin  = 1e-6,
    Dmax  = 1e-2,
    N0 =  8e6,
    useFuncArgDefaults = True,
)


Note that there is a warning in case not all arguments are used

In [ ]:
pam2.describeHydrometeor(
    pamtra2.hydrometeors.softEllipsoidMassSize,
    name='snow',
    nBins = 20,
    sizeBounds=pamtra2.hydrometeors.size.logspaceBounds, #function/object to call for getting sizes
    sizeCenter = pamtra2.hydrometeors.size.boundsToMid,
    sizeBoundsWidth = pamtra2.hydrometeors.size.boundsWidth,
    sizeDistribution = pamtra2.hydrometeors.sizeDistribution.exponentialFieldWC,
    aspectRatio = 0.6,
    crossSectionArea = pamtra2.hydrometeors.crossSectionArea.powerLaw,
    mass = pamtra2.hydrometeors.mass.powerLaw,
    density = pamtra2.hydrometeors.density.softEllipsoid,
    #to do: adapt refractiveIndex routine so that mixing and RI are treated separately. 
    relativePermittivity = pamtra2.libs.refractiveIndex.snow.eps,
    scattering = pamtra2.hydrometeors.scattering.Mie,
    fallVelocity = pamtra2.hydrometeors.fallVelocity.heymsfield10_particles,
    Dmin = 1e-6,
    Dmax = 1e-2,
    massSizeA = 0.0121, 
    massSizeB = 1.9,
    areaSizeA = 0.4,
    areaSizeB = 1.8,
    minDensity = 100,
    bla = 12,
    useFuncArgDefaults = True,
    )


We have access to a lot of internal properties, like the fall velocity of particles

In [ ]:
plt.plot(
    pam2.hydrometeors.rain.profile.sizeCenter,
    pam2.hydrometeors.rain.profile.fallVelocity.isel(layer=5,time=0),
    label = 'rain')
plt.plot(
    pam2.hydrometeors.snow.profile.sizeCenter,
    pam2.hydrometeors.snow.profile.fallVelocity.isel(layer=5,time=0),
    label = 'snow')
plt.legend()
plt.grid()


In [ ]:
plt.plot(
    pam2.hydrometeors.rain.profile.sizeCenter,
    pam2.hydrometeors.rain.profile.backscatterCrossSection.isel(layer=5,time=0,frequency=0),
    label = 'rain')
plt.plot(
    pam2.hydrometeors.snow.profile.sizeCenter,
    pam2.hydrometeors.snow.profile.backscatterCrossSection.isel(layer=5,time=0,frequency=0),
    label = 'snow')
plt.legend()
plt.grid()


The profile arrays have been populated, but they contain daks arrays. These are only placeholders, they have not been calculated yet!

In [ ]:
pam2.hydrometeors.rain.profile, pam2.hydrometeors.snow.profile

Force computation with e.g.
`pam2.hydrometeors.rain.profile.compute()` or `pam2.hydrometeors.rain.profile.to_netcdf()` or  or `pam2.hydrometeors.rain.profile.fallVelocity.plot()`

# Radar simulator


In [ ]:


pam2.addInstrument(
    pamtra2.instruments.radar.dopplerRadarPamtra,
    name = 'test',
    frequencies = 3.5e+10,    
        radarMaxV=7.885,
        radarMinV=-7.885,
        radarAliasingNyquistInterv=1,
        radarNFFT=256,
        verbosity=0,
        radarAirmotion=True,
        radarAirmotionModel="constant",  # "constant","linear","step"
        radarAirmotionVmin=0,
        radarAirmotionVmax=0,
        radarAirmotionLinearSteps=30,
        radarAirmotionStepVmin=0.5,
        radarK2=0.93,         # dielectric constant |K|² (always for liquid water by convention) for the radar equation
        radarBeamwidthDeg=0.2,
        radarIntegrationTime=60,
        radarPNoise1000=-30,
        radarNAve=150,
        seed=0,
)

Again, the results are not estimated yet when using dask.

In [ ]:
pam2.instruments.test.results


Do the numbers!

In [ ]:
pam2.instruments.test.results.load()

In [ ]:
pam2.instruments.test.results.radarReflectivity.isel(peak=0,frequency=0).plot(y='layer')

